In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math

In [31]:
d_model = 10
max_word = 4
position = torch.arange(0,max_word)
print(position.shape)
print(position)
position = torch.arange(0,max_word).float().unsqueeze(1)
print(position.shape)
print(position)
exponent = torch.arange(0,d_model,2)/d_model
angle = 1/torch.pow(1000,exponent).float()
pe = torch.zeros(max_word, d_model).float()
pe.require_grad = False

pe[:, 0::2] = torch.sin(position/angle)
pe[:, 1::2] = torch.cos(position/angle)
print(type(pe))

torch.Size([4])
tensor([0, 1, 2, 3])
torch.Size([4, 1])
tensor([[0.],
        [1.],
        [2.],
        [3.]])
<class 'torch.Tensor'>


In [5]:
torch.pow(1000,exponent)

tensor([   1.0000,    3.9811,   15.8489,   63.0957,  251.1887, 1000.0000])

In [33]:
max_len = 4
d_model = 10
pe = torch.zeros(max_len, d_model).float()
pe.require_grad = False

position = torch.arange(0, max_len).float().unsqueeze(1)
div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)
print(type(pe))
print(pe.shape)

<class 'torch.Tensor'>
torch.Size([4, 10])


In [26]:
(math.log(10000.0) / d_model).exp()

AttributeError: 'float' object has no attribute 'exp'

In [34]:
torch.tensor([1,1,1])

tensor([1, 1, 1, 1])

In [76]:

class PositionalEmbed(nn.Module):

    def __init__(self,embed_d,max_len=4000):
        super(PositionalEmbed,self).__init__()
        angle = self.get_angle(embed_d,max_len)
        pe = torch.zeros(max_len, embed_d)
        pe.requires_grad = False
        pe[:,0::2] = torch.sin(angle)
        print(pe[:,1::2].shape)
        pe[:,1::2] = torch.cos(angle)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe',pe)

        

    def get_angle(self, embed_d, max_len):
        with torch.no_grad():
            pos = torch.arange(0,max_len).float().unsqueeze(1)
            print(pos.shape)
            angle = 1 / (torch.pow(10000.0,torch.arange(0,embed_d,2)/embed_d))

        return  pos * angle


    def forward(self, x):
        print(self.pe.shape)
        return x + self.pe[:,:x.size(1)]

In [91]:
x = torch.tensor([[[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]]]).float()
print(x.shape)
posEmb_l = PositionalEmbed(4,4)
out= posEmb_l(x)
print(out.shape)

torch.Size([1, 4, 4])
torch.Size([4, 1])
torch.Size([4, 2])
torch.Size([1, 4, 4])
torch.Size([1, 4, 4])


In [50]:
torch.arange(0,4,2)

tensor([0, 2])

In [208]:
class AttentionLayer(nn.Module):

    def __init__(self, n_heads, embed_d,mask=None):
        super(AttentionLayer, self).__init__()
        self.mask = mask
        self.d_k = embed_d//n_heads
        print(self.d_k)
        self.W_q = nn.Linear(embed_d,embed_d//n_heads)#nn.Parameter(torch.ones(embed_d,self.d_k)) #nn.Linear(embed_d,embed_d//n_heads)
        self.W_k = nn.Linear(embed_d,embed_d//n_heads)#nn.Parameter(torch.ones(embed_d,self.d_k)) #nn.Linear(embed_d,embed_d//n_heads)
        self.W_v = nn.Linear(embed_d,embed_d//n_heads)#nn.Parameter(torch.ones(embed_d,self.d_k)) #nn.Linear(embed_d,embed_d//n_heads)


    def forward(self, x):
        # Q = torch.matmul(x,self.W_q)
        # K = torch.matmul(x,self.W_k)
        # V = torch.matmul(x,self.W_v)
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)
        K_t = K.transpose(2,1)
        print(K_t.shape)
        
        logit = torch.matmul(Q,K_t)/math.sqrt(self.d_k)

        #masking 한 영역을 지운다고 생각하면 됨 softmax 결과 0이 되버림
        if self.mask is not None:
            logit += (self.mask * -1e9)

        attn_weight = F.softmax(logit,dim=1)
        attn_score = torch.matmul(attn_weight,V)

        return attn_score,attn_weight


In [209]:
att_l = AttentionLayer(2,4)
out,_ = att_l(x)
print(out)

2
torch.Size([1, 2, 4])
tensor([[[-1.4785, -0.3787],
         [-0.9148, -0.2621],
         [-0.5847, -0.1903],
         [-0.3875, -0.1447]]], grad_fn=<UnsafeViewBackward0>)


In [8]:
root = 'F:\\data\\ETDataset\\ETT-small'
from data.dataLoader import Dataset_ETT_hour


In [9]:
etthset = Dataset_ETT_hour(root)

UnboundLocalError: local variable 'df_data' referenced before assignment

In [1]:
if 'S' in ['s']:
    print('ok')

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'